# canny, gaussian blur <- 이미지 노이즈 줄이기

- hsv 색상 표 : https://colorpicker.me/#585ba1

In [2]:
import os, cv2, math
import numpy as np

In [40]:
# path = './medinode_data_001_120/001_649900540_아빌리파이정1밀리그램/000.jpg'
# 한글 경로때문에 기존 cv2.imread(path) 는 사용 불가능!!
# 그래서 해결방법은 한글 경로가 포함된 경우
# numpy 행렬로 읽은 다음 cv2.imdecode 를 통해 복호화 해주면 opencv에서 사용가능
# a = np.fromfile(path, np.uint8)

# img = cv2.imdecode(a, cv2.IMREAD_COLOR)

# path = './medinode_data_001_120/002/005.jpg'
path = './medinode_data_001_120/015/001.jpg'
img = cv2.imread(path)

# 이미지 정보
img_w, img_h, img_c = img.shape

if img is None:
    print('경로 확인 필요')

# 윤곽선 잡기
# canny
one = 50
two = 900
img_canny = cv2.Canny(img, one, two)

# # gaussian blur 표준편차 사용하여 이미지 균일하게 다룸
img_gaussian = cv2.GaussianBlur(img, ksize=(0,0), sigmaX=0.1)
img_gaussian_canny = cv2.Canny(img_gaussian, one, two)

# # 외곽선 검출 함수 cv2.findContours
img_count = cv2.findContours(img_gaussian_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # 외곽선 검출 그리기 cv2.drawContours
# draw = cv2.drawContours(img, contours=img_count[0][0],contourIdx=-1, color=(0,0,255), thickness=2)
draw = cv2.drawContours(img, contours=img_count[0],contourIdx=-1, color=(0,0,255), thickness=2)

# 조건문 넣어서 여기에 박스 
# cv2.rectangle(img, 시작점, 종료점, color=색상, thickness=두께)

# cv2.imshow('org', img)
# cv2.imshow('canny', img_canny)
# cv2.imshow('gaussian', img_gaussian)
cv2.imshow('gaussian_canny', img_gaussian_canny)
# cv2.imshow('draw', draw)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# 윤곽선 검출시 총 개수
len(img_count[0])

4

In [ ]:
# 첫번째 탐지된 객체의 각 점의 좌표 <- 이거 이용해서 최대 x, y 축 찾아낼수있음 그치만 방향은 알기 어렵.
img_count[0][0]

array([[[509, 222]],

       [[508, 223]],

       [[508, 224]],

       [[507, 225]],

       [[507, 227]],

       [[509, 227]],

       [[510, 226]],

       [[510, 223]],

       [[511, 222]],

       [[510, 223]]], dtype=int32)

In [ ]:
len(img_count)

2

In [11]:
# 위에서 얻은 윤곽선 데이터를 이용해서 각 점의 x, y값 구하기
# 그러면 가장 max_x, min_x, max_y, min_y 좌표를 얻어서 탐지된 박스의 넓이 구한 후
# 넓이가 5 이하면 알약이 아니므로 그건 생략하고 나머지 객체들을 박스 지정!

# 현재는 박스 하나만 선택하고 넓이 구하고 
point_x = []
point_y = []
for i in img_count[0][0]:
    for x, y in i:
        point_x.append(x)
        point_y.append(y)

len(point_x), len(point_y)

(10, 10)

In [12]:
# x좌표
max_x = max(point_x)
min_x = min(point_x)

max_x, min_x

(511, 507)

In [13]:
# y좌표
max_y = max(point_y)
min_y = min(point_y)

max_x, min_y

(511, 222)

In [14]:
len_x = abs(max_x - min_x)
len_y = abs(max_y - min_y)

len_x, len_y

# 박스 넓이
box_area = len_x * len_y
box_area

20

In [72]:
import os, cv2, math
import numpy as np

# 흰색 타원형
# path = './medinode_data_001_120/001/001.jpg'

# 흰색 캡슐
# path = './medinode_data_001_120/015/001.jpg'

# 녹색 타원형 <- 흰색으로 잡힘
# path = './medinode_data_001_120/004/001.jpg'

# 파란색 타원형 (3개잡힙)
path = './medinode_data_001_120/005/000.jpg'

# 노란색 원형 <- 흰색으로 잡힘
# path = './medinode_data_001_120/013/001.jpg'

# 흰색 + 빨강 캡슐
# path = './medinode_data_001_120/007/001.jpg'


img = cv2.imread(path)

if img is None:
    print('경로 확인 필요')
    
# 이미지 정보
img_w, img_h, img_c = img.shape

# 윤곽선 잡기
# canny
one = 60
two = 900
img_canny = cv2.Canny(img, one, two)

# # gaussian blur 표준편차 사용하여 이미지 균일하게 다룸
img_gaussian = cv2.GaussianBlur(img, ksize=(0,0), sigmaX=0.1)
img_gaussian_canny = cv2.Canny(img_gaussian, one, two)

# # 외곽선 검출 함수 cv2.findContours
img_count = cv2.findContours(img_gaussian_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(len(img_count[0]))
# img_count는 윤곽선의 정보를 담고있다.
# 기존 자료형이 튜플여서 수정불가. 그래서 리스트로 변환
ls_count = list(img_count[0])

if len(ls_count) < 3:
    raise('윤곽선 검출 실패!!! canny 쓰레스홀드 수정 필요')
else:
    drop_index = []
    index = 0
    img_count[0]
    for i in img_count[0]:
        point_x = []
        point_y = []    
        
        # 리스트 차원축소 안하면 for문 하나더 사용해야함.
        ls_2d = sum(i.tolist(), [])
        for x, y in ls_2d:
            point_x.append(x)
            point_y.append(y)
        
        # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
        max_x = max(point_x)
        min_x = min(point_x)
        max_y = max(point_y)
        min_y = min(point_y)   

        # 점과 점사이의 거리 이용해서 x, y 구하기
        len_x = abs(max_x - min_x)
        len_y = abs(max_y - min_y)

        # 현재는 알약 500 < 넓이 < 100000 면 알약이 아닌것으로 판단
        box_size = len_x * len_y
        if box_size < 500:
            drop_index.append(index)
            index += 1
        elif 100000 < box_size:
            drop_index.append(index)
            index += 1 
        else:
            index += 1

    drop_index.reverse()
    # 불필요한 객체 삭제
    for _ in drop_index:
        ls_count.pop(_)

# 박스 좌표 구하기
box_point = []
tag = 0
for ls in ls_count:
    p_x = []
    p_y = []
    ls_2d = sum(ls.tolist(), [])
    for x, y in ls_2d:
        p_x.append(x)
        p_y.append(y)
        
    # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
    max_x = max(p_x)
    min_x = min(p_x)
    max_y = max(p_y)
    min_y = min(p_y)

    # 박스의 각 좌표
    # 왼쪽위 x1, 오른쪽위 x2, 왼쪽아래 x3, 오른쪽아래 x4
    x1 = (min_x - 10, min_y - 10)       # 파라미터
    x2 = (max_x + 10, min_y - 10)
    x3 = (min_x - 10, max_y + 10)
    x4 = (max_x + 10, max_y + 10)
    box_point.append((x1,x4))
    # 원본이미지에 알약 박스 그리기 - 색상 녹색
    cv2.rectangle(img, x1, x4, color=(0,255,0), thickness=2)

    # 각 알약 다른 창으로 시각화
    pill = img[min_y -1 : max_y + 10, min_x -10 : max_x + 10]
    window_name_hsv = f'pill {tag}'
    window_name_hsv_convert = f'hsv_convert {tag}'
    window_name_hsv_color = f'hsv_color {tag}'

    # rgb -> hsv로 변환
    hsv_img = cv2.cvtColor(pill, cv2.COLOR_BGR2HSV)

    # hsv(색공간/채도/명도)
    min_white = np.array([0, 0, 200])
    max_white = np.array([360, 255, 255])

    min_red = np.array([0, 100, 100])
    max_red = np.array([10, 255, 255])
    
    min_green = np.array([30, 40, 40])
    max_green = np.array([90, 255, 255])

    min_blue = np.array([100, 100, 100])
    max_blue = np.array([120, 255, 255])

    min_yellow = np.array([20, 100, 100])
    max_yellow = np.array([30, 255, 255])

    min_pink = np.array([140, 50, 50])
    max_pink = np.array([180, 255, 255])

    # 색상 검출
    hsv_show_white = cv2.inRange(hsv_img, min_white, max_white)
    hsv_show_red = cv2.inRange(hsv_img, min_red, max_red)
    hsv_show_green = cv2.inRange(hsv_img, min_green, max_green)
    hsv_show_blue = cv2.inRange(hsv_img, min_blue, max_blue)
    hsv_show_yellow = cv2.inRange(hsv_img, min_yellow, max_yellow)
    hsv_show_pink = cv2.inRange(hsv_img, min_pink, max_pink)

    # 색상 검출된거에서 이미지에서 해당 색상있으면 하얀색이므로 
    # countNonZero 함수 사용해서 빠르게 검출
    white = cv2.countNonZero(hsv_show_white)
    red = cv2.countNonZero(hsv_show_red)
    green = cv2.countNonZero(hsv_show_green)
    blue = cv2.countNonZero(hsv_show_blue)
    yellow = cv2.countNonZero(hsv_show_yellow)
    pink = cv2.countNonZero(hsv_show_pink)

    top_color = [white, red, green, blue, yellow, pink]
    print(top_color)
    ls_color = ['white', 'red', 'green', 'blue', 'yellow', 'pink']
    find_color = top_color.index(max(top_color))
    
    # 해당 알약에서 최종 색상
    re_color = ls_color[find_color]
    print(re_color)
    
    # 탐지된 알약 시각화
    cv2.imshow(window_name_hsv, pill)
    # cv2.imshow(window_name_hsv_convert, hsv_img)
    # cv2.imshow(window_name_hsv_color, hsv_show_white) # inrange 한 결과보기
    tag += 1


# # 외곽선 검출 그리기 cv2.drawContours
draw = cv2.drawContours(img, contours=img_count[0][4],contourIdx=-1, color=(0,0,255), thickness=2)
# draw = cv2.drawContours(img, contours=ls_count,contourIdx=-1, color=(0,0,255), thickness=2)

cv2.imshow('draw', draw)

cv2.waitKey()
cv2.destroyAllWindows()

6
[4039, 0, 351, 4462, 0, 0]
blue
[5014, 0, 380, 5172, 0, 0]
blue
[379, 0, 378, 1395, 0, 0]
blue


In [70]:
img_count[0][4]

array([[[ 86,  95]],

       [[ 78, 103]],

       [[ 78, 104]],

       ...,

       [[ 76, 106]],

       [[ 85,  97]],

       [[ 85,  96]]], dtype=int32)

In [62]:
for _ in img_count:
    print()
    for a in _:
        a

[[[254 312]]

 [[253 313]]

 [[246 313]]

 [[245 314]]

 [[242 314]]

 [[241 315]]

 [[238 315]]

 [[237 316]]

 [[234 316]]

 [[233 317]]

 [[231 317]]

 [[230 318]]

 [[228 318]]

 [[227 319]]

 [[225 319]]

 [[224 320]]

 [[223 320]]

 [[222 321]]

 [[221 321]]

 [[220 322]]

 [[218 322]]

 [[216 324]]

 [[214 324]]

 [[213 325]]

 [[212 325]]

 [[210 327]]

 [[209 327]]

 [[208 328]]

 [[208 329]]

 [[207 330]]

 [[206 330]]

 [[205 331]]

 [[203 331]]

 [[199 335]]

 [[198 335]]

 [[194 339]]

 [[193 339]]

 [[189 343]]

 [[189 344]]

 [[187 346]]

 [[187 347]]

 [[186 348]]

 [[186 350]]

 [[185 351]]

 [[185 355]]

 [[186 356]]

 [[186 359]]

 [[187 360]]

 [[187 362]]

 [[188 363]]

 [[188 365]]

 [[189 366]]

 [[189 367]]

 [[190 368]]

 [[190 370]]

 [[191 371]]

 [[191 372]]

 [[192 373]]

 [[192 374]]

 [[194 376]]

 [[194 377]]

 [[196 379]]

 [[197 379]]

 [[198 380]]

 [[200 380]]

 [[201 381]]

 [[215 381]]

 [[216 380]]

 [[219 380]]

 [[220 379]]

 [[225 379]]

 [[226

In [33]:
sum((img_count[0])[0].tolist(), [])

[[509, 222],
 [508, 223],
 [508, 224],
 [507, 225],
 [507, 227],
 [509, 227],
 [510, 226],
 [510, 223],
 [511, 222],
 [510, 223]]

색상표(hsv)<br>
https://myyac.tistory.com/133<br>
참고 사이트<br>
https://bradbury.tistory.com/64

In [15]:
#
min_blue = (200, 100, 100)
max_blue = (255, 100, 100)

path  = './red_blue.jpg'

# path = './medinode_data_001_120/001/000.jpg'
img = cv2.imread(path)
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# bgr
bgr_min_red = (0, 0, 200)
bgr_max_red = (0, 0, 255)
bgr_show_red = cv2.inRange(img, bgr_min_red, bgr_max_red)

# hsv(색공간/채도/명도)
# 0에 가까울수록 빨강
min_red = np.array([0, 0, 100])
max_red = np.array([10, 255, 255])
hsv_show_red = cv2.inRange(hsv_img, min_red, max_red)

min_white = np.array([0, 0, 230])
max_white = np.array([360, 255, 255])
hsv = cv2.inRange(hsv_img, min_white, max_white)

red_count = cv2.countNonZero(hsv_show_red)
# print(red_count)
print(hsv)
cv2.imshow('org', img)
# cv2.imshow('hsv', hsv_img)
# cv2.imshow('bgr_red', bgr_show_red)
cv2.imshow('hsv_red', hsv)

cv2.waitKey()
cv2.destroyAllWindows()

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [10]:
check = []
color = 0
for i in hsv_show_red:
    # print(len(i))
    for _ in i:
        # print(_)
        if _ > 0:
            color += 1
            check.append(color)

In [12]:
color

1

In [ ]:
check = []
color = 0
for i in hsv_show_red:
    # print(len(i))
    for _ in i:
        # print(_)
        if _ > 0:
            color += 1
            check.append(color)

18289


In [29]:
import cv2

# 이미지 읽기
path  = './red_blue.jpg'
image = cv2.imread(path)

# BGR -> HSV 변환
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 파란색 범위 (하한, 상한)
min_blue = np.array([100, 100, 100])
max_blue = np.array([120, 255, 255])

# 파란색 영역 추출
hsv_show_blue = cv2.inRange(hsv, min_blue, max_blue)

# 결과 이미지 출력
cv2.imshow("Result", hsv_show_blue)
cv2.waitKey(0)
cv2.destroyAllWindows()


# 버전1 - 이미지에서 윤곽선사용하여 알약 추출 후 각각의 알약 다른 화면으로 보여줌.

In [38]:
import os, cv2, math
import numpy as np

path = './medinode_data_001_120/015/001.jpg'

img = cv2.imread(path)

if img is None:
    print('경로 확인 필요')
    
# 이미지 정보
img_w, img_h, img_c = img.shape

# 윤곽선 잡기
# canny
one = 50
two = 900
img_canny = cv2.Canny(img, one, two)

# # gaussian blur 표준편차 사용하여 이미지 균일하게 다룸
img_gaussian = cv2.GaussianBlur(img, ksize=(0,0), sigmaX=0.1)
img_gaussian_canny = cv2.Canny(img_gaussian, one, two)

# # 외곽선 검출 함수 cv2.findContours
img_count = cv2.findContours(img_gaussian_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# img_count는 윤곽선의 정보를 담고있다.
# 기존 자료형이 튜플여서 수정불가. 그래서 리스트로 변환
ls_count = list(img_count[0])

if len(ls_count) < 3:
    raise('윤곽선 검출 실패!!! canny 쓰레스홀드 수정 필요')
else:
    drop_index = []
    index = 0
    img_count[0]
    for i in img_count[0]:
        point_x = []
        point_y = []    
        
        # 리스트 차원축소 안하면 for문 하나더 사용해야함.
        ls_2d = sum(i.tolist(), [])
        for x, y in ls_2d:
            point_x.append(x)
            point_y.append(y)
        
        # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
        max_x = max(point_x)
        min_x = min(point_x)
        max_y = max(point_y)
        min_y = min(point_y)   

        # 점과 점사이의 거리 이용해서 x, y 구하기
        len_x = abs(max_x - min_x)
        len_y = abs(max_y - min_y)

        # 현재는 알약 500 < 넓이 < 100000 면 알약이 아닌것으로 판단
        box_size = len_x * len_y
        if box_size < 500:
            drop_index.append(index)
            index += 1
        elif 100000 < box_size:
            drop_index.append(index)
            index += 1 
        else:
            index += 1

    drop_index.reverse()
    # 불필요한 객체 삭제
    for _ in drop_index:
        ls_count.pop(_)

# 박스 좌표 구하기
box_point = []
tag = 0
for ls in ls_count:
    p_x = []
    p_y = []
    ls_2d = sum(ls.tolist(), [])
    for x, y in ls_2d:
        p_x.append(x)
        p_y.append(y)
        
    # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
    max_x = max(p_x)
    min_x = min(p_x)
    max_y = max(p_y)
    min_y = min(p_y)

    # 박스의 각 좌표
    # 왼쪽위 x1, 오른쪽위 x2, 왼쪽아래 x3, 오른쪽아래 x4
    x1 = (min_x - 10, min_y - 10)
    x2 = (max_x + 10, min_y - 10)
    x3 = (min_x - 10, max_y + 10)
    x4 = (max_x + 10, max_y + 10)
    box_point.append((x1,x4))
    # 원본이미지에 알약 박스 그리기 - 색상 녹색
    cv2.rectangle(img, x1, x4, color=(0,255,0), thickness=2)

    # 각 알약 다른 창으로 시각화
    pill = img[min_y -1 : max_y + 10, min_x -10 : max_x + 10]
    window_name_hsv = f'pill {tag}'
    window_name_hsv_color = f'hsv_color {tag}'
    # cv2.imshow(window_name, pill)

    # rgb -> hsv로 변환
    hsv_img = cv2.cvtColor(pill, cv2.COLOR_BGR2HSV)

    # hsv(색공간/채도/명도)
    min_white = np.array([0, 0, 200])
    max_white = np.array([360, 255, 255])

    min_red = np.array([0, 100, 100])
    max_red = np.array([10, 255, 255])
    
    min_green = ()
    max_green = ()

    min_blue = ()
    max_blue = ()

    min_yellow = ()
    max_yellow = ()

    # white
    hsv_show_white = cv2.inRange(hsv_img, min_white, max_white)
    # red
    hsv_show_red = cv2.inRange(hsv_img, min_red, max_red)

    # cv2.imshow(window_name_hsv, hsv_img)
    cv2.imshow(window_name_hsv_color, hsv_show_white)
    tag += 1


# # 외곽선 검출 그리기 cv2.drawContours
draw = cv2.drawContours(img, contours=ls_count,contourIdx=-1, color=(0,0,255), thickness=2)

cv2.imshow('draw', img)

cv2.waitKey()
cv2.destroyAllWindows()

# 버전2 - result 색상, 알약 검출 완료

In [ ]:
import os, cv2, math
import numpy as np

# img 경로
path = './medinode_data_001_120/015/001.jpg'

img = cv2.imread(path)

if img is None:
    print('경로 확인 필요')
    
# 이미지 정보
img_w, img_h, img_c = img.shape

# 윤곽선 잡기
# canny
one = 50
two = 900
img_canny = cv2.Canny(img, one, two)

# # gaussian blur 표준편차 사용하여 이미지 균일하게 다룸
img_gaussian = cv2.GaussianBlur(img, ksize=(0,0), sigmaX=0.1)
img_gaussian_canny = cv2.Canny(img_gaussian, one, two)

# # 외곽선 검출 함수 cv2.findContours
img_count = cv2.findContours(img_gaussian_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# img_count는 윤곽선의 정보를 담고있다.
# 기존 자료형이 튜플여서 수정불가. 그래서 리스트로 변환
ls_count = list(img_count[0])

if len(ls_count) < 3:
    raise('윤곽선 검출 실패!!! canny 쓰레스홀드 수정 필요')
else:
    drop_index = []
    index = 0
    img_count[0]
    for i in img_count[0]:
        point_x = []
        point_y = []    
        
        # 리스트 차원축소 안하면 for문 하나더 사용해야함.
        ls_2d = sum(i.tolist(), [])
        for x, y in ls_2d:
            point_x.append(x)
            point_y.append(y)
        
        # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
        max_x = max(point_x)
        min_x = min(point_x)
        max_y = max(point_y)
        min_y = min(point_y)   

        # 점과 점사이의 거리 이용해서 x, y 구하기
        len_x = abs(max_x - min_x)
        len_y = abs(max_y - min_y)

        # 현재는 알약 500 < 넓이 < 100000 면 알약이 아닌것으로 판단
        box_size = len_x * len_y
        if box_size < 500:
            drop_index.append(index)
            index += 1
        elif 100000 < box_size:
            drop_index.append(index)
            index += 1 
        else:
            index += 1

    drop_index.reverse()
    # 불필요한 객체 삭제
    for _ in drop_index:
        ls_count.pop(_)

# 박스 좌표 구하기
box_point = []
tag = 0
for ls in ls_count:
    p_x = []
    p_y = []
    ls_2d = sum(ls.tolist(), [])
    for x, y in ls_2d:
        p_x.append(x)
        p_y.append(y)
        
    # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
    max_x = max(p_x)
    min_x = min(p_x)
    max_y = max(p_y)
    min_y = min(p_y)

    # 박스의 각 좌표
    # 왼쪽위 x1, 오른쪽위 x2, 왼쪽아래 x3, 오른쪽아래 x4
    x1 = (min_x - 10, min_y - 10)
    x2 = (max_x + 10, min_y - 10)
    x3 = (min_x - 10, max_y + 10)
    x4 = (max_x + 10, max_y + 10)
    box_point.append((x1,x4))
    # 원본이미지에 알약 박스 그리기 - 색상 녹색
    cv2.rectangle(img, x1, x4, color=(0,255,0), thickness=2)

    # 각 알약 다른 창으로 시각화
    pill = img[min_y -1 : max_y + 10, min_x -10 : max_x + 10]
    window_name_hsv = f'pill {tag}'
    window_name_hsv_convert = f'hsv_convert {tag}'
    window_name_hsv_color = f'hsv_color {tag}'

    # rgb -> hsv로 변환
    hsv_img = cv2.cvtColor(pill, cv2.COLOR_BGR2HSV)

    # hsv(색공간/채도/명도)
    min_white = np.array([0, 0, 200])
    max_white = np.array([360, 255, 255])

    min_red = np.array([0, 100, 100])
    max_red = np.array([10, 255, 255])
    
    min_green = np.array([30, 40, 40])
    max_green = np.array([90, 255, 255])

    min_blue = np.array([100, 100, 100])
    max_blue = np.array([120, 255, 255])

    min_yellow = np.array([20, 100, 100])
    max_yellow = np.array([30, 255, 255])

    min_pink = np.array([140, 50, 50])
    max_pink = np.array([180, 255, 255])

    # 색상 검출
    hsv_show_white = cv2.inRange(hsv_img, min_white, max_white)
    hsv_show_red = cv2.inRange(hsv_img, min_red, max_red)
    hsv_show_green = cv2.inRange(hsv_img, min_green, max_green)
    hsv_show_blue = cv2.inRange(hsv_img, min_blue, max_blue)
    hsv_show_yellow = cv2.inRange(hsv_img, min_yellow, max_yellow)
    hsv_show_pink = cv2.inRange(hsv_img, min_pink, max_pink)

    # 색상 검출된거에서 이미지에서 해당 색상있으면 하얀색이므로 
    # countNonZero 함수 사용해서 빠르게 검출
    white = cv2.countNonZero(hsv_show_white)
    red = cv2.countNonZero(hsv_show_red)
    green = cv2.countNonZero(hsv_show_green)
    blue = cv2.countNonZero(hsv_show_blue)
    yellow = cv2.countNonZero(hsv_show_yellow)
    pink = cv2.countNonZero(hsv_show_pink)

    top_color = [white, red, green, blue, yellow, pink]
    ls_color = ['white', 'red', 'green', 'blue', 'yellow', 'pink']
    find_color = top_color.index(max(top_color))
    
    # 해당 알약에서 최종 색상
    re_color = ls_color[find_color]
    print(re_color)
    
    # 탐지된 알약 시각화
    cv2.imshow(window_name_hsv, pill)
    # cv2.imshow(window_name_hsv_convert, hsv_img)
    # cv2.imshow(window_name_hsv_color, hsv_show_white) # inrange 한 결과보기
    tag += 1


# # 외곽선 검출 그리기 cv2.drawContours
draw = cv2.drawContours(img, contours=ls_count,contourIdx=-1, color=(0,0,255), thickness=2)

cv2.imshow('draw', draw)

cv2.waitKey()
cv2.destroyAllWindows()